Notebook for sampling a diffusion model trained on mnist data

Sources:

[The Network Code](https://github.com/FluxML/model-zoo/blob/master/vision/diffusion_mnist/diffusion_mnist.jl)

[The Sampling Code](https://github.com/FluxML/model-zoo/blob/master/vision/diffusion_mnist/diffusion_plot.jl)

[Useful Explainer](https://yang-song.net/blog/2021/score/)

[Original Paper](https://arxiv.org/pdf/2011.13456.pdf)

Notes:
- Sampling is in general slow (~1h for 1000steps for mcmc sampler) (and cannot run on gpu). Make sampling a job to be submitted?

In [26]:
# Dependencies
using Pkg
Pkg.activate(".")
include("unet_large.jl")
include("diffusion_sampling.jl")
using MLDatasets
using Plots
using Random
using CUDA
using Statistics
using ProgressMeter
using ProgressMeter: Progress, next!
using BSON


  Activating project at `/gpfs/work1/0/einf220/bathy_machine_learning/julia_ml_tests.jl.git`


In [2]:
model = BSON.load("models/diffusion_fullunet_mnist60k_epoch50.bson")[:model]

UNET((fourier_proj = GaussFourierProjection{Vector{Float32}}(Float32[1.9219507, 20.89066, 48.726906, -56.949436, -67.83617, -6.370674, -31.336685, 3.1547246, 26.389194, -23.954872  …  26.294327, 19.657393, 16.10706, -4.456802, 70.16656, 58.152153, -19.021622, -13.773496, -52.536335, -32.086838]), linear = Dense(256 => 256, swish), conv1 = Conv((3, 3), 1 => 32, bias=false), dense1 = Dense(256 => 32), gnorm1 = GroupNorm(32, 4, swish), conv2 = Conv((3, 3), 32 => 64, pad=1, stride=2, bias=false), dense2 = Dense(256 => 64), gnorm2 = GroupNorm(64, 32, swish), conv3 = Conv((3, 3), 64 => 128, pad=1, stride=2, bias=false), dense3 = Dense(256 => 128), gnorm3 = GroupNorm(128, 32, swish), conv4 = Conv((3, 3), 128 => 256, pad=1, stride=2, bias=false), dense4 = Dense(256 => 256), gnorm4 = GroupNorm(256, 32, swish), tconv4 = ConvTranspose((3, 3), 256 => 128, pad=1, stride=2, bias=false), dense5 = Dense(256 => 128), tgnorm4 = GroupNorm(128, 32, swish), tconv3 = ConvTranspose((3, 3), 256 => 64, pad=1, 

Test whether loading has worked. The fourier_proj layer is typically where it fails

In [3]:
model.layers.fourier_proj([0,1])

256×2 Matrix{Float32}:
  0.0  -0.470977
  0.0  -0.634223
  0.0  -0.989492
 -0.0   0.312363
 -0.0   0.856989
 -0.0  -0.726063
 -0.0  -0.855298
  0.0   0.826107
  0.0   0.641317
 -0.0   0.279755
  ⋮    
  1.0  -0.549587
  1.0   0.782155
  1.0  -0.963391
  1.0   0.500573
  1.0   0.576776
  1.0   0.990785
  1.0   0.147094
  1.0  -0.974048
  1.0   0.854801

## Setup sampler parameters

In [ ]:
t = ones(Float32, 5) |> gpu
# (randn(Float32, (28,28,1,5)).*
#         expand_dims(marginal_prob_std(t),3)) |> gpu

In [4]:
device = gpu
time_steps, Δt, init_x = setup_sampler(device, 5, 2000)

(LinRange{Float32}(1.0f0, 0.001f0, 2000), 0.00049972534f0, [10.841808 -10.871333 … 2.4578469 -17.094824; -2.2217932 1.1885667 … 9.215737 7.2395964; … ; -0.37808245 2.1751199 … 2.787806 5.136489; 8.961721 -7.754802 … 6.9379935 -3.4797456;;;; -8.013173 -11.220552 … -22.56979 -10.20969; 10.505686 9.399887 … -3.8172708 -15.341513; … ; -14.537037 -6.6486535 … -8.036965 5.172932; -0.11098013 -28.144342 … -6.5835047 -12.782855;;;; -1.1887243 3.104252 … -18.099434 6.1798086; -8.01442 -6.780413 … 9.337905 -16.364897; … ; 10.665331 1.003793 … 5.4984417 7.159678; -3.9915042 9.930873 … 3.0624576 -11.764578;;;; 5.9394813 2.7645528 … 10.004532 -5.758919; -2.2725153 1.5973158 … -0.4522584 6.452466; … ; -10.445722 -8.748006 … 15.22096 3.0581152; -0.40628198 -3.6529481 … -7.1957674 3.8338132;;;; 4.774878 -21.236252 … -19.054146 2.4248915; 1.5776793 -1.305831 … 6.1437078 -2.5042784; … ; -2.9022822 -13.341394 … 7.8973637 8.99367; -3.3497171 -4.839087 … -2.8529022 -2.3641212])

## Euler-Maruyama sampler

In [ ]:
length(time_steps)

In [5]:
em = EM_sampler(model |> device, init_x, time_steps, Δt);

Progress:   0%|                                         |  ETA: 9:58:09



Progress:  50%|████████████████████▌                    |  ETA: 0:00:40



Progress:  52%|█████████████████████▍                   |  ETA: 0:00:37



Progress:  54%|██████████████████████                   |  ETA: 0:00:34



Progress:  56%|██████████████████████▊                  |  ETA: 0:00:32


...


Progress:  94%|██████████████████████████████████████▊  |  ETA: 0:00:02



Progress:  96%|███████████████████████████████████████▌ |  ETA: 0:00:02



Progress:  98%|████████████████████████████████████████▎|  ETA: 0:00:01



Progress: 100%|████████████████████████████████████████▉|  ETA: 0:00:00



Progress: 100%|█████████████████████████████████████████| Time: 0:00:42
  time_step:  2000


In [6]:
em_sample = ConvertToImage(em |> cpu ,size(em)[end])

## MCMC sampler

In [7]:
mcmc = MCMC_sampler(model |> device, init_x, time_steps, Δt)

Progress:   0%|                                         |  ETA: 1:06:34



Progress:   1%|▍                                        |  ETA: 0:07:07



Progress:   2%|▊                                        |  ETA: 0:03:49



Progress:   3%|█▏                                       |  ETA: 0:02:40



Progress:   4%|█▍                                       |  ETA: 0:02:03


...


Progress:  98%|████████████████████████████████████████ |  ETA: 0:00:00



Progress:  99%|████████████████████████████████████████▍|  ETA: 0:00:00



Progress:  99%|████████████████████████████████████████▊|  ETA: 0:00:00



Progress: 100%|█████████████████████████████████████████| Time: 0:00:17
  time_step:  2000


28×28×1×5 CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}:
[:, :, 1, 1] =
 -0.0313584    -0.00372765  -0.0163342    …   0.0175372   -0.0138768
  0.0144195    -0.0315875   -0.00167096      -0.0115456   -0.0129122
  0.0646378     0.00670059   0.0037215       -0.0291886   -0.00271729
 -0.0215029     0.0203031   -0.00299159       0.00151461   0.000290482
 -0.0225113    -0.00930709   0.0224183        0.0318987    0.0129611
 -0.00657533   -0.00890062   0.011752     …   0.0225109    0.011732
 -0.00476225   -0.0502921   -0.00162268       0.0251469   -0.0222397
 -0.018289      7.38769f-5   0.00663213       0.005129    -0.000552648
  0.000763684   0.0011788    0.00872701      -0.0236274    0.000347391
  0.0452024     0.0200636    0.0107823        0.0216616    0.00739004
  ⋮                                       ⋱               
  0.00205968    0.0033404    0.000547763      0.00310963  -0.000579459
  0.0129003    -0.023825     0.0359405    …  -0.00776377   0.00242542
  0.00236233    0.0251609   -0.000

In [8]:
mcmc_sample = ConvertToImage(mcmc |> cpu,size(mcmc)[end])

## Tests Inpainting

In [10]:
using MLDatasets

In [12]:
n_ims = 10
xtest, _ = MNIST(:test)[rand(1:10000, n_ims)]
xtest = reshape(xtest, (28,28,1,:));

In [18]:
function set_blocks(x, nblanks)
    nrows = size(x)[1]
    ncols = size(x)[2]
    nimages = size(x)[4]
    
    mask = zeros(Float32, size(x))

    for i = 1:nimages
        blank_rows = rand(1:nblanks)
        start_row = rand(1:(nrows-blank_rows))
        end_row = start_row + blank_rows
        blank_cols = rand(1:nblanks)
        start_col = rand(1:(ncols-blank_cols))
        end_col = start_col + blank_cols
        mask[start_row:end_row, start_col:end_col,:,i] .= 1
    end
    x_out = (1 .- mask).*x
    return x_out, mask
end

set_blocks (generic function with 2 methods)

In [19]:
nblanks = 20
masked, masks = set_blocks(xtest, nblanks)
ConvertToImage(masked, size(masked)[end])

In [22]:
device = gpu
time_steps, Δt, init_x = setup_sampler(device, n_ims, 2000)

masked = masked |> device
masks = masks |> device

28×28×1×10 CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0

In [24]:
em_inpaint = EM_inpainter(model |> device, masked, masks, init_x, time_steps, Δt);

In [25]:
ConvertToImage(em_inpaint |> cpu, size(em_inpaint)[end])

In [27]:
mcmc_inpaint = MCMC_inpainter(model |> device, masked, masks, init_x, time_steps, Δt)

28×28×1×10 CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0       …  0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0       …  0.176471  0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.905882  0.176471  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.992157  0.470588  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.992157  0.882353  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.992157  0.992157  0.0  0.0  0.0
 ⋮                        ⋮         ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.607843     0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.4       …  0.0       0.0     

In [28]:
ConvertToImage(mcmc_inpaint |> cpu, size(mcmc_inpaint)[end])